Este notebook realiza uma validação em um agrupamento de dados diferente, ainda pertencente ao dataset COCO, utilizando o grupo de validação do COCO 2014.

Isso permite avaliar a capacidade de generalização do modelo em dados não vistos durante o treinamento, mas que fazem parte do mesmo universo do COCO.

In [1]:
# Instalação de Dependências

!pip install -q tqdm torchmetrics opencv-python
!pip install -U albumentations

print("Dependências prontas.")

Dependências prontas.


In [2]:
# Imports Globais

# --- PyTorch e Torchvision ---
import torch
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torch.utils.data import Dataset, DataLoader

# --- Manipulação de Imagens e Dados ---
from PIL import Image  	# Biblioteca para abrir e manipular imagens (Pillow).
import numpy as np
import albumentations as A	# Biblioteca para data augmentation.
from pycocotools.coco import COCO	# Utilitário para manusear anotações COCO.
from albumentations.pytorch import ToTensorV2	# Converte imagens (numpy/pil) para tensores PyTorch.

# --- Utilitários de Treinamento e Avaliação ---
from tqdm.auto import tqdm	# Cria barras de progresso que funcionam bem em notebooks.
from torchmetrics.detection.mean_ap import MeanAveragePrecision  # Métrica padrão para detecção de objetos.
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision.models.detection.rpn import AnchorGenerator
from torchvision.models.detection import FasterRCNN
from torchvision.transforms import functional as F # torchvision para usar a função to_tensor.

# --- Bibliotecas Padrão do Python ---
import os 		# Para manipulação de caminhos e arquivos.
import glob  	# Para encontrar arquivos
import re  		# Para usar expressões regulares
import random	# Para operações de aleatoriedade
import cv2  	# OpenCV, para desenhar caixas e texto nas imagens de teste.
import matplotlib.pyplot as plt # Para exibir imagens no notebook.

In [3]:
# Célula 4: Configuração

# Define uma classe para centralizar todas as configurações do projeto.
class Config:
    
    # --- CAMINHO PARA O DATASET ---
    DATA_ROOT_PATH = '/kaggle/input/coco-dataset/my_data'
    
    # --- CONFIGURAÇÕES GERAIS ---
    # Define o dispositivo de computação: 'cuda' (GPU) se disponível, senão 'cpu'.
    DEVICE = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
    
    # Diretório onde os checkpoints do modelo serão salvos durante o treinamento.
    CHECKPOINT_DIR = '/kaggle/working/checkpoints/'
    
    # Define o número total de classes. Como o modelo precisa de uma classe para o fundo (background),
    # o número é (quantidade de classes reais + 1).
    NUM_CLASSES = 4  # 3 classes (person, car, dog) + 1 background
    
    # Lista com os nomes das categorias que queremos extrair do dataset COCO.
    CATEGORIAS_DESEJADAS = ['person', 'car', 'dog']
    
    # --- CONFIGURAÇÕES DO DATASET ---
    # Limita o número de imagens para acelerar o treinamento e a validação.
    IMAGE_LIMIT = 3000  	# Limite para o conjunto de treino.
    VAL_IMAGE_LIMIT = 300 	# Limite para o conjunto de validação.

    # Caminhos específicos para os dados de treino e validação e seus arquivos de anotação JSON.
    TRAIN_DATA_DIR = os.path.join(DATA_ROOT_PATH, 'train/')
    TRAIN_COCO = os.path.join(DATA_ROOT_PATH, 'annotations/instances_train2017.json')
    VAL_DATA_DIR = os.path.join(DATA_ROOT_PATH, 'val/')
    VAL_COCO = os.path.join(DATA_ROOT_PATH, 'annotations/instances_val2017.json')

    # --- HIPERPARÂMETROS DE TREINAMENTO ---
    NUM_EPOCHS = 20  		# Número total de épocas para treinar o modelo.
    TRAIN_BATCH_SIZE = 1 	# Quantidade de imagens por lote de treinamento.
    TRAIN_SHUFFLE_DL = True	# Embaralhar o dataset de treino a cada época.
    NUM_WORKERS_DL = 0  	# Número de processos para carregar dados. 0 significa que será na thread principal.

    # Parâmetros do otimizador SGD (Gradiente Descendente Estocástico).
    LR = 0.001  	# Taxa de aprendizado (learning rate).
    MOMENTUM = 0.9  # Momento, ajuda a acelerar o SGD na direção certa.
    WEIGHT_DECAY = 0.0005  # Termo de regularização L2 para evitar overfitting.

# Cria uma instância da classe de configuração para ser usada no restante do código.
config = Config()
print("Configurações definidas.")

Configurações definidas.


In [4]:
# ==============================================================================
# CÉLULA 5: FUNÇÕES UTILITÁRIAS (VERSÃO FINAL E CORRIGIDA)
# Esta versão contém a correção definitiva para o problema de tipo de dado.
# ==============================================================================

# Classe customizada de Dataset para o formato COCO, com filtros.
class FilteredCOCODataset(Dataset):
    
    def __init__(self, root, annotation, transforms=None, cats=None, limit=None):
        
        self.root = root  	# Diretório das imagens.
        self.transforms = transforms  # Transformações de data augmentation.
        self.coco = COCO(annotation)  # Carrega o arquivo de anotações COCO.
        
        # Pega os IDs numéricos das categorias desejadas (ex: 'person', 'car').
        self.desired_cat_ids = set(self.coco.getCatIds(catNms=cats if cats else []))
        
        # Mapeia os IDs originais do COCO para os IDs do nosso modelo (1, 2, 3...).
        self.coco_to_model_map = {coco_id: i + 1 for i, coco_id in enumerate(sorted(list(self.desired_cat_ids)))}
        
        # Lógica para limitar e balancear o dataset.
        if limit and limit > 0 and len(self.desired_cat_ids) > 0:
            
            # Obtém uma lista de imagens para cada categoria desejada.
            imgs_per_cat = {cat_id: self.coco.getImgIds(catIds=[cat_id]) for cat_id in self.desired_cat_ids}
            
            # Calcula quantas imagens pegar por categoria para atingir o limite.
            limit_per_cat = int(limit / len(self.desired_cat_ids))
            balanced_img_ids = set() 	# Usa um conjunto para evitar duplicatas.
            
            for cat_id in self.desired_cat_ids:
                
                image_list = imgs_per_cat[cat_id]
                random.shuffle(image_list)  # Embaralha para pegar uma amostra aleatória.
                balanced_img_ids.update(image_list[:limit_per_cat])
                
            final_ids = list(balanced_img_ids)
            random.shuffle(final_ids) # Embaralha a lista final de IDs.
            
            # Garante que não passamos do limite e ordena os IDs.
            self.ids = sorted(final_ids[:limit])
            
        else:
            
            # Se não houver limite, pega todas as imagens que contêm as categorias desejadas.
            all_image_ids = set()
            
            for cat_id in self.desired_cat_ids:
                all_image_ids.update(self.coco.getImgIds(catIds=[cat_id]))
                
            self.ids = list(sorted(list(all_image_ids)))

    # Método que carrega e retorna um único item (imagem e anotações) do dataset.
    def __getitem__(self, idx):
        
        img_id = self.ids[idx] # Pega o ID da imagem pelo índice.
        
        try:
            
            # Carrega informações da imagem e suas anotações do COCO.
            ann_ids = self.coco.getAnnIds(imgIds=img_id)
            coco_annotations = self.coco.loadAnns(ann_ids)
            path = self.coco.loadImgs(img_id)[0]['file_name']
            img = Image.open(os.path.join(self.root, path)).convert('RGB')
            
            boxes, labels = [], []
            
            for obj in coco_annotations:
                
                # Filtra apenas os objetos das categorias desejadas e com área válida.
                if obj['category_id'] in self.desired_cat_ids and obj['bbox'][2] > 0 and obj['bbox'][3] > 0:
                    
                    boxes.append(obj['bbox']) # Bbox no formato [x, y, width, height].
                    labels.append(self.coco_to_model_map[obj['category_id']]) # Usa o label mapeado.
            
            if not boxes: return None # Pula a imagem se não tiver objetos de interesse.

            # Converte as bboxes para o formato [xmin, ymin, xmax, ymax] exigido pelo PyTorch.
            boxes_tensor = torch.as_tensor(boxes, dtype=torch.float32).reshape(-1, 4)
            boxes_tensor[:, 2:] += boxes_tensor[:, :2]
            
            # Cria o dicionário 'target' com as anotações formatadas.
            target = {'boxes': boxes_tensor, 'labels': torch.as_tensor(labels, dtype=torch.int64)}

            # Aplica as transformações (data augmentation).
            if self.transforms:
                
                transformed = self.transforms(image=np.array(img), bboxes=target['boxes'].numpy(), labels=target['labels'].numpy())
                img = transformed['image']
                if len(transformed['bboxes']) == 0: return None 	# Pula se a augmentação removeu todas as bboxes.
                target['boxes'] = torch.as_tensor(transformed['bboxes'], dtype=torch.float32).reshape(-1, 4)
                target['labels'] = torch.as_tensor(transformed['labels'], dtype=torch.int64)
            
            # --- SOLUÇÃO DEFINITIVA ---
            # Garante que a imagem seja um tensor de ponto flutuante (float) e normalizada para o intervalo [0, 1].
            # ToTensorV2 deveria fazer isso, mas essa conversão manual garante a consistência.
            if not img.is_floating_point():
                img = img.to(torch.float32) / 255.0

            return img, target
        
        except Exception:
            
            # Retorna None se houver qualquer erro ao processar a imagem (ex: arquivo corrompido).
            return None

    # Método que retorna o número total de amostras no dataset.
    def __len__(self):
        
        return len(self.ids)

# Função que define a sequência de transformações de imagem.
def get_transform(train):
    
    transforms_list = []
    bbox_params = None
    
    if train:
        
        # Se for para treinamento, aplica data augmentation.
        transforms_list.append(A.HorizontalFlip(p=0.5)) # Espelhamento horizontal com 50% de chance.
        transforms_list.append(A.RandomBrightnessContrast(p=0.2)) # Muda brilho e contraste.
        
        # Define os parâmetros para as bounding boxes, para que elas se ajustem com as augmentações.
        bbox_params = A.BboxParams(format='pascal_voc', label_fields=['labels'], min_area=1, min_visibility=0.1)
    
    # Sempre converte a imagem para um tensor PyTorch.
    transforms_list.append(ToTensorV2())
    
    # Compõe (agrupa) as transformações em um único pipeline.
    if bbox_params:
        return A.Compose(transforms_list, bbox_params=bbox_params)
    
    else:
        return A.Compose(transforms_list)

# Função de agrupamento para o DataLoader.
def collate_fn(batch):
    
    # Filtra amostras que retornaram 'None' do __getitem__.
    batch = [b for b in batch if b is not None]
    
    if not batch:
        return None, None
    
    # Separa as imagens e os alvos em duas tuplas separadas.
    return tuple(zip(*batch))

# Função para criar o modelo de detecção.
def get_detection_model(num_classes):
    
    # Carrega um modelo Faster R-CNN com backbone ResNet50, pré-treinado no COCO.
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn_v2(weights="DEFAULT")
    
    # Obtém o número de características de entrada do classificador.
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    
    # Substitui a "cabeça" do classificador por uma nova, com o número correto de classes para nosso problema.
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
    return model

print("Célula 5: Funções Utilitárias definidas (VERSÃO FINAL E CORRIGIDA).")

Célula 5: Funções Utilitárias definidas (VERSÃO FINAL E CORRIGIDA).


In [5]:
# ==============================================================================
# CÉLULA 6: ENGINE DE TREINAMENTO (VERSÃO CORRIGIDA)
# ==============================================================================

# Função para treinar o modelo por uma época.
def train_one_epoch(model, optimizer, data_loader, device, epoch):
    """
    Executa uma única época de treinamento. (Esta função já estava correta)
    """
    
    model.train()  # Coloca o modelo em modo de treinamento.
    prog_bar = tqdm(data_loader, total=len(data_loader), desc=f"Época {epoch+1} [Treino]")
    train_epoch_loss = 0
    
    # Itera sobre os lotes de dados do data_loader de treinamento.
    for i, data in enumerate(prog_bar):
        
        if data is None or data[0] is None:
            continue  # Pula o lote se for inválido.
            
        imgs, annotations = data
        
        # Move as imagens e anotações para o dispositivo (GPU/CPU).
        imgs = list(img.to(device) for img in imgs)
        annotations = [{k: v.to(device) for k, v in t.items()} for t in annotations]
        
        # O modelo retorna um dicionário de perdas (losses) quando está em modo de treino.
        loss_dict = model(imgs, annotations)
        
        # Soma todas as perdas (ex: perda de classificação, perda de regressão da caixa).
        losses = sum(loss for loss in loss_dict.values())
        
        # Verificação de segurança: se a perda for infinita ou NaN, pula a atualização.
        if not torch.isfinite(losses):
            print(f"ALERTA: Loss infinita na iteração {i}, pulando batch.")
            continue

        optimizer.zero_grad()   # Zera os gradientes acumulados.
        losses.backward()  		# Calcula os gradientes (backpropagation).
        
        # Limita a norma dos gradientes para evitar "exploding gradients".
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()	# Atualiza os pesos do modelo.
        
        train_epoch_loss += losses.item()  		 	# Acumula a perda da época.
        prog_bar.set_postfix(loss=losses.item())	# Atualiza a barra de progresso com a loss atual.

    # Retorna a média da perda de treinamento da época.
    return train_epoch_loss / len(data_loader) if len(data_loader) > 0 else 0.0


# --- FUNÇÃO 'evaluate' ---
# Decorador que desativa o cálculo de gradientes, economizando memória e acelerando a execução.
@torch.no_grad()
def evaluate(model, data_loader, device):
    """
    Executa la evaluación del modelo en el conjunto de datos de validación.
    """
    model.eval()	# Coloca o modelo em modo de avaliação.
    
    prog_bar = tqdm(data_loader, total=len(data_loader), desc="[Validação]")
    validation_loss = 0
    
    # Itera sobre os lotes de dados do data_loader de validação.
    for i, data in enumerate(prog_bar):
        if data is None or data[0] is None:
            continue

        imgs, annotations = data
        imgs = list(img.to(device) for img in imgs)
        annotations = [{k: v.to(device) for k, v in t.items()} for t in annotations]
        
        was_training = model.training	# Guarda o estado atual (que é 'eval').
        model.train()	# Muda para modo 'train' para obter o dict de loss.
        loss_dict = model(imgs, annotations)
        model.train(was_training)		# Restaura o estado original ('eval').

        losses = sum(loss for loss in loss_dict.values())
        
        if torch.isfinite(losses): 
            validation_loss += losses.item()
        
        prog_bar.set_postfix(loss=losses.item())
        
    # Retorna a média da perda de validação.
    return validation_loss / len(data_loader) if len(data_loader) > 0 else 0.0

print("Célula 6: Funções de Engine definidas (com 'evaluate' corrigido).")

Célula 6: Funções de Engine definidas (com 'evaluate' corrigido).


In [6]:

# --- Função para criar o modelo (versão específica para ResNet-50) ---
def get_model_fasterrcnn_resnet50(num_classes):
    """
    Carrega um modelo Faster R-CNN pré-treinado (backbone ResNet-50)
    e o adapta para o número de classes desejado.
    """
    
    # Carrega o modelo com pesos pré-treinados.
    model = fasterrcnn_resnet50_fpn(weights='FasterRCNN_ResNet50_FPN_Weights.DEFAULT')
    
    # Obtém o número de features da camada de predição.
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    
    # Substitui a camada de predição por uma nova, adequada ao nosso número de classes.
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
    return model

In [7]:
# --- Função para criar o modelo ---
def get_model_fasterrcnn_mobilenet(num_classes):
    
    """
    Cria um modelo Faster R-CNN usando um backbone MobileNetV2.
    Este é um modelo mais leve que o ResNet-50.
    """
    
    # Carrega o extrator de features (backbone) do MobileNetV2 pré-treinado no ImageNet.
    backbone = torchvision.models.mobilenet_v2(weights='MobileNet_V2_Weights.DEFAULT').features
    
    # O número de canais de saída do backbone é necessário para a próxima camada.
    backbone.out_channels = 1280
    
    # Define o gerador de âncoras para a Region Proposal Network (RPN).
    anchor_generator = AnchorGenerator(sizes=((32, 64, 128, 256, 512),), aspect_ratios=((0.5, 1.0, 2.0),))
    
    # Define a camada de RoI pooling.
    roi_pooler = torchvision.ops.MultiScaleRoIAlign(featmap_names=['0'], output_size=7, sampling_ratio=2)
    
    # Constrói o modelo Faster R-CNN com os componentes definidos.
    model = FasterRCNN(backbone, num_classes=num_classes, rpn_anchor_generator=anchor_generator, box_roi_pool=roi_pooler)
    return model

In [8]:

# --- Função para criar o modelo ---
def get_model_fasterrcnn_efficientnet(num_classes):
    """
    Cria um modelo Faster R-CNN usando um backbone EfficientNet-B0.
    """
    
    # Carrega o extrator de features (backbone) do EfficientNet-B0 pré-treinado.
    backbone = torchvision.models.efficientnet_b0(weights='EfficientNet_B0_Weights.DEFAULT').features
    
    # Define manualmente o número de canais de saída do backbone.
    backbone.out_channels = 1280
    
    # Define o gerador de âncoras e o RoI Pooler, similar ao MobileNet.
    anchor_generator = AnchorGenerator(sizes=((32, 64, 128, 256, 512),), aspect_ratios=((0.5, 1.0, 2.0),))
    roi_pooler = torchvision.ops.MultiScaleRoIAlign(featmap_names=['0'], output_size=7, sampling_ratio=2)
    
    # Constrói o modelo Faster R-CNN final.
    model = FasterRCNN(backbone, num_classes=num_classes, rpn_anchor_generator=anchor_generator, box_roi_pool=roi_pooler)
    
    return model


In [9]:
# =========================
# Célula 11: Validação em Novo Agrupamento COCO (validação no grupo de validação do COCO 2014)
# Este código realiza a validação dos modelos treinados usando um subconjunto (801 imagens) do grupo de validação do COCO 2014 (val2014).
# =========================

NOVO_DATA_DIR = '/kaggle/input/val2014/val2014'
NOVO_COCO = '/kaggle/input/coco-image-caption/annotations_trainval2014/annotations/instances_val2014.json'

# Cria o novo dataset e dataloader para validação (usando 801 imagens)
novo_dataset = FilteredCOCODataset(
    root=NOVO_DATA_DIR,
    annotation=NOVO_COCO,
    transforms=get_transform(train=False),
    cats=config.CATEGORIAS_DESEJADAS,
    limit=500
)
novo_data_loader = DataLoader(
    novo_dataset,
    batch_size=1,
    shuffle=False,
    num_workers=config.NUM_WORKERS_DL,
    collate_fn=collate_fn
)
print(f"Novo agrupamento: {len(novo_dataset)} imagens.\n")

# Lista de modelos para validar (cada dicionário contém o nome, caminho do modelo e função de criação)
modelos_info = [
    {
        "nome": "ResNet50",
        "caminho": "/kaggle/input/fastercnn_resnet50/tensorflow2/default/1/best_model_checkpoint.pth",
        "funcao": get_model_fasterrcnn_resnet50  # ou get_model_fasterrcnn_resnet50 se for esse o nome
    },
    {
        "nome": "EfficientNetB0",
        "caminho": "/kaggle/input/fastercnn_efficientnetb0/tensorflow2/default/1/best_model_checkpoint.pth",
        "funcao": get_model_fasterrcnn_efficientnet
    },
    {
        "nome": "MobileNetV2",
        "caminho": "/kaggle/input/fastercnn_mobilenetv2/tensorflow2/default/1/best_model.pth",
        "funcao": get_model_fasterrcnn_mobilenet
    }
]

# Loop para validar cada modelo no grupo de validação do COCO 2014
for info in modelos_info:
    print(f"=== Validação para {info['nome']} ===")
    best_model_path = info["caminho"]
    modelo = info["funcao"](config.NUM_CLASSES)
    checkpoint = torch.load(best_model_path, map_location=config.DEVICE)
    modelo.to(config.DEVICE)

    # Avaliação do loss
    loss_novo_agrupamento = evaluate(modelo, novo_data_loader, config.DEVICE)
    print(f"Loss no novo agrupamento: {loss_novo_agrupamento:.4f}")


loading annotations into memory...
Done (t=12.77s)
creating index...
index created!
Novo agrupamento: 497 imagens.

=== Validação para ResNet50 ===


[Validação]:   0%|          | 0/497 [00:00<?, ?it/s]

Loss no novo agrupamento: 2.4393
=== Validação para EfficientNetB0 ===


[Validação]:   0%|          | 0/497 [00:00<?, ?it/s]

Loss no novo agrupamento: 2.3135
=== Validação para MobileNetV2 ===


[Validação]:   0%|          | 0/497 [00:00<?, ?it/s]

Loss no novo agrupamento: 2.2242
